 # TIME SERIES SALES FORCASTING - REGRESSION
 
## BUSINESS UNDERSTANDING 

### INTRODUCTION

Time series forecasting is one of the key topics of machine learning. The fact that so many prediction issues have a temporal component makes it crucial. In contrast to many other prediction tasks, time series issues are more challenging since the time component contributes more information.

#### TIME SERIES FORCASTING APPLICATION 
Time series forecasting is employed in various sectors, including finance, supply chain management, production, and inventory planning, making it one of the most widely used data science approaches. Time series forecasting has many applications, including resource allocation, business planning, weather forecasts, and stock price prediction.

Time series forecasting can be used by any business or organization dealing with continuously generated data and the requirement to adjust to operational shifts and changes. Here, machine learning acts as the greatest enabler, improving our ability to:

#### Web traffic forecasting: 
In order to forecast online traffic rates during certain periods, common data on typical traffic rates among competing websites is combined with input data on traffic-related trends.

#### Sales and demand forecasting: 
Customer behavior pattern data, in combination with inputs from purchase history, demand history, seasonal influence, etc., enables machine learning models to identify the most demanded items and pinpoint their placement in the dynamic market.

#### Weather prediction: 
Time-based data is routinely collected from a variety of globally networked weather stations, and machine learning approaches enable in-depth analysis and interpretation of the data for future forecasts based on statistical dynamics.

#### Stock price forecasting: 
In order to make accurate forecasts of the most likely impending stock price movements, one can integrate historical stock price data with information on regular and atypical spikes and decreases in the stock market.

#### Economic and demographic forecasting: 
Demographics and economics have a ton of statistical data that can be utilized to forecast time series data effectively. Consequently, the ideal target market can be determined, and the most effective strategies to communicate with that specific TA may be developed.

#### Academics: 
The concepts of machine learning and deep learning greatly speed up the processes of refining and launching scientific ideas. For instance, scientific data that must go through infinite analytical cycles may be analyzed considerably more quickly with machine learning patterns.

### PROJECT OBJECTIVE
In this project, we'll predict store sales on data from Corporation Favorita, a large Ecuadorian-based grocery retailer.

We shall also build a model that will accurately predicts the unit sales for thousands of items sold at different Favorita stores.

### HYPOTHESIS STATEMENT
Null Hypothesis:Sales are not affected by promotions, oil prices and holidays. 
Alternate Hypothesis: Sales are affected by promotions, oil prices and holidays

### ANALYTICAL QUESTIONS

1) Is the train dataset complete (has all the required dates)?

2) Which dates have the lowest and highest sales for each year?

3) Did the earthquake impact sales?

4) Are certain groups of stores selling more products? (Cluster, city, state, type)

5) Are sales affected by promotions, oil prices and holidays?

6) What analysis can we get from the date and its extractable features?

7) What is the difference between RMSLE, RMSE, MSE (or why is the MAE greater than all of them?)


## IMPORTING RELEVANT LIBRARIES

In [38]:
import pandas as pd
import numpy as np
import pyodbc    
from dotenv import dotenv_values

import statsmodels.api as sm

##Visualization Libraries 
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import seaborn as sns 
import random
import plotly.offline as offline
offline.init_notebook_mode(connected=True) # Configure Plotly to run offline


# Feature Processing libraries
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif


#Algorithms libraries
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import train_test_split
import xgboost

##pipelines and transformers 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer 
from imblearn.pipeline import Pipeline
from imblearn.combine import SMOTEENN
from sklearn.pipeline import make_pipeline

##handling imbalance datasets
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight

##hyperparameter tuning
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

##for hypothesis testing 
from scipy.stats import chi2_contingency

##model evaluation:
from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score

# Other packages
import os
from itertools import product
import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")

# DATA UNDERSTANDING


## LOADING ALL DATASETS

Firstly, we shall fetch some datasets from sql server using the method below:

In [2]:
# Load environment variables from .env file into a dictionary
environment_variables = dotenv_values('.env')

# Get the values for the credentials you set in the '.env' file
database = environment_variables.get("DATABASE")
server = environment_variables.get("SERVER")
username = environment_variables.get("USERNAME")
password=environment_variables.get("PASSWORD")

connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"

In [3]:
# Use the connect method of the pyodbc library and pass in the connection string.
#This will connect to the server
connection = pyodbc.connect(connection_string)

In [4]:
# Now the sql query to get the 1st data 
query = "Select * from dbo.oil"
oil = pd.read_sql(query,connection)

#oil.to_csv('oil.csv')  

oil

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.139999
2,2013-01-03,92.970001
3,2013-01-04,93.120003
4,2013-01-07,93.199997
...,...,...
1213,2017-08-25,47.650002
1214,2017-08-28,46.400002
1215,2017-08-29,46.459999
1216,2017-08-30,45.959999


In [5]:
# Now the sql query to get the 2nd data 
query = "Select * from dbo.holidays_events"
holidays = pd.read_sql(query,connection)

#holidays.to_csv('holidays.csv')  

holidays

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


In [6]:
# Now the sql query to get the 3rd data 
query = "Select * from dbo.stores"
stores = pd.read_sql(query,connection)

#stores.to_csv('stores.csv')  

stores

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4
5,6,Quito,Pichincha,D,13
6,7,Quito,Pichincha,D,8
7,8,Quito,Pichincha,D,8
8,9,Quito,Pichincha,B,6
9,10,Quito,Pichincha,C,15


In [12]:
sample_submission = pd.read_csv("sample_submission.csv")

sample_submission

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0
...,...,...
28507,3029395,0.0
28508,3029396,0.0
28509,3029397,0.0
28510,3029398,0.0


In [13]:
test = pd.read_csv("test.csv")

test

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


In [14]:
transactions = pd.read_csv("transactions.csv")

transactions

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922
...,...,...,...
83483,2017-08-15,50,2804
83484,2017-08-15,51,1573
83485,2017-08-15,52,2255
83486,2017-08-15,53,932


In [16]:
train = pd.read_excel("train.xlsx")

train

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,1,2013-01-01,1,BABY CARE,0.000,0
2,2,2013-01-01,1,BEAUTY,0.000,0
3,3,2013-01-01,1,BEVERAGES,0.000,0
4,4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...,...
1048570,1048570,2014-08-13,3,POULTRY,974.098,1
1048571,1048571,2014-08-13,3,PREPARED FOODS,324.293,0
1048572,1048572,2014-08-13,3,PRODUCE,10.000,1
1048573,1048573,2014-08-13,3,SCHOOL AND OFFICE SUPPLIES,0.000,0


## Exploratory Data Analysis (EDA)

In [17]:
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.139999
2,2013-01-03,92.970001
3,2013-01-04,93.120003
4,2013-01-07,93.199997


In [18]:
holidays.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [19]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [21]:
sample_submission.head()

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0


In [22]:
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [23]:
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [24]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [25]:
# creating a list of the datasets

list_datasets = [stores, train, test, transactions, oil, holidays, sample_submission]

In [26]:
# creating a function that will Check for the info on all the datasets

def general_info(datasets):
    for data in datasets:
        variable_name  = globals()
        print(data.info())
#         print('\n')
        print('_' * 50)

In [27]:
general_info(list_datasets)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB
None
__________________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 6 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   id           1048575 non-null  int64         
 1   date         1048575 non-null  datetime64[ns]
 2   store_nbr    1048575 non-null  int64         
 3   family       1048575 non-null  object        
 4   sales        1048575 non-null  float64       
 5   onpromotion  1048575 non-null  int64         

In [28]:
# creating a function that will Check for the missing values on all the datasets

def show_missing_val(datasets):
    for data in datasets:
        print(data.isnull().sum())
        print('_' * 50)

In [29]:
show_missing_val(list_datasets)

store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64
__________________________________________________
id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64
__________________________________________________
id             0
date           0
store_nbr      0
family         0
onpromotion    0
dtype: int64
__________________________________________________
date            0
store_nbr       0
transactions    0
dtype: int64
__________________________________________________
date           0
dcoilwtico    43
dtype: int64
__________________________________________________
date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64
__________________________________________________
id       0
sales    0
dtype: int64
__________________________________________________


Only the oil dataset has missing values.

## ANSWERING SOME OF THE ANALYTICAL QUESTIONS

### 1. Is the train dataset complete (has all the required dates)?

In [30]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [31]:
# CONVERTING THE DATE TO DATE TIME

train['date'] = pd.to_datetime(train['date'])

In [32]:
# DETERMINING THE MIN AND MAX TIME
train['date'].min(), train['date'].max()

(Timestamp('2013-01-01 00:00:00'), Timestamp('2014-08-13 00:00:00'))

In [37]:
# Extracting dates which are not in the train dataset

missing_dates=pd.date_range(start= '2013-01-01', end='2014-08-13').difference(train.date)

missing_dates

DatetimeIndex(['2013-12-25'], dtype='datetime64[ns]', freq=None)

from the above, the train dataset does not have all the required date. The missing date is '2013-12-25'.

### TREATING THE MISSING DATE

In [44]:
#to fix missing dates in the train dataset, we would import iterable product
#this will help us to loop over each item in the selected arguments 

#creating variables as arguments for the product() method

missing_dates
uniques_stores=train.store_nbr.unique()
unique_family=train.family.unique()

In [45]:
#we will replace the missing dates by pairing it with all the unique stores and families

replace_dates=list(product(missing_dates, uniques_stores,unique_family ))

In [46]:
#creating a dataframe for the replaced dates

replace_dates_df=pd.DataFrame(replace_dates, columns=['date', 'store_nbr', 'family'])

replace_dates_df.head()

,date,store_nbr,family
0,2013-12-25,1,AUTOMOTIVE
1,2013-12-25,1,BABY CARE
2,2013-12-25,1,BEAUTY
3,2013-12-25,1,BEVERAGES
4,2013-12-25,1,BOOKS


In [47]:
#adding replaced dates to our train data
new_train=pd.concat([train, replace_dates_df], ignore_index=True)

new_train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0.0,2013-01-01,1,AUTOMOTIVE,0.0,0.0
1,1.0,2013-01-01,1,BABY CARE,0.0,0.0
2,2.0,2013-01-01,1,BEAUTY,0.0,0.0
3,3.0,2013-01-01,1,BEVERAGES,0.0,0.0
4,4.0,2013-01-01,1,BOOKS,0.0,0.0


In [49]:
missing_dates=pd.date_range(start= '2013-01-01', end='2014-08-13').difference(new_train.date)

missing_dates

DatetimeIndex([], dtype='datetime64[ns]', freq=None)

Now,there is no more missing dates.

In [50]:
new_train.isnull().sum()

id             1782
date              0
store_nbr         0
family            0
sales          1782
onpromotion    1782
dtype: int64

In [51]:
#filling Nan values in sales and onpromotion column with 0

new_train.fillna(0,inplace=True)

new_train.isnull().sum()


id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64

In [52]:
#dropping id column since it's not relevant

new_train=new_train.drop('id', axis='columns')

new_train.head()

,date,store_nbr,family,sales,onpromotion
0,2013-01-01,1,AUTOMOTIVE,0.0,0.0
1,2013-01-01,1,BABY CARE,0.0,0.0
2,2013-01-01,1,BEAUTY,0.0,0.0
3,2013-01-01,1,BEVERAGES,0.0,0.0
4,2013-01-01,1,BOOKS,0.0,0.0


## 2. Which dates have the lowest and highest sales for each year?

In [53]:
# Making a copy of the new_train and storing it as train2

train2 = new_train.copy()

train2['Year'] = train2['date'].dt.year

In [54]:
train2.head()

,date,store_nbr,family,sales,onpromotion,Year
0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,2013
1,2013-01-01,1,BABY CARE,0.0,0.0,2013
2,2013-01-01,1,BEAUTY,0.0,0.0,2013
3,2013-01-01,1,BEVERAGES,0.0,0.0,2013
4,2013-01-01,1,BOOKS,0.0,0.0,2013


In [85]:
# Calculating the aggregate Sales 

aggregate_sales = train2.groupby(['date']).agg(daily_sales=('sales', 'sum'), year=('Year', 'mean'))

aggregate_sales.head()

,daily_sales,year
date,,
2013-01-01,2511.618999,2013.0
2013-01-02,496092.417944,2013.0
2013-01-03,361461.231124,2013.0
2013-01-04,354459.677093,2013.0
2013-01-05,477350.121229,2013.0


In [86]:

aggregate_sales['year'] = aggregate_sales['year'].astype(int).astype(str)

In [87]:
aggregate_sales.head()

,daily_sales,year
date,,
2013-01-01,2511.618999,2013
2013-01-02,496092.417944,2013
2013-01-03,361461.231124,2013
2013-01-04,354459.677093,2013
2013-01-05,477350.121229,2013


In [88]:
# Creating a function that will calculate the min_max sales for each year 

def min_max_sales(aggregate):
    unique_years = aggregate['year'].unique().tolist()
    for year in unique_years:
        year_agg = aggregate.loc[str(year)]
        max_sale = year_agg['daily_sales'].max()
        min_sale = year_agg['daily_sales'].min()
        print(f'The maximum and minimum sales in {year} are ${max_sale:,.2f} and ${min_sale:,.2f} respectively')

In [89]:
min_max_sales(aggregate_sales)

The maximum and minimum sales in 2013 are $792,865.28 and $0.00 respectively
The maximum and minimum sales in 2014 are $949,618.79 and $8,602.07 respectively


## 3. Did the earthquake impact sales?

### A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.

In [90]:
new_train.head()

,date,store_nbr,family,sales,onpromotion
0,2013-01-01,1,AUTOMOTIVE,0.0,0.0
1,2013-01-01,1,BABY CARE,0.0,0.0
2,2013-01-01,1,BEAUTY,0.0,0.0
3,2013-01-01,1,BEVERAGES,0.0,0.0
4,2013-01-01,1,BOOKS,0.0,0.0


In [93]:
# Extracting two weeks sales before the earthquake

start_date='2016-04-01'

end_date='2016-04-15'

#setting date to datetime

new_train['date']=pd.to_datetime(new_train['date'])

# getting start and end date using mask function to select rows between dates

mask = (new_train['date']>=start_date) & (new_train['date']<=end_date)

start_end_df_b4=new_train.loc[mask]

start_end_df_b4.head()

,date,store_nbr,family,sales,onpromotion


In [94]:
start_end_df_b4

,date,store_nbr,family,sales,onpromotion
